In [56]:
!pip install evaluate
!pip install transformers
!pip install datasets

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [57]:
from typing import List, Tuple

In [65]:
from datasets import load_dataset, Dataset, concatenate_datasets
import evaluate
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline

In [59]:
MODEL_NAME = 'deepvk/USER-bge-m3'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [132]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

In [109]:
data_from_kaggle = pd.read_csv('/kaggle/input/news-dataset-for-lab/ds.csv')[::25]
new_data = {
    'index_id': list(range(2005, 2005 + len(data_from_kaggle))),
    'category': list(data_from_kaggle['category']),
    'text': list(data_from_kaggle['title'])
}

new_dataset = Dataset.from_dict(new_data)

train_set = concatenate_datasets([train_set, new_dataset])

In [133]:
train_set

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 701
})

In [134]:
MINIBATCH_SIZE = 16
tokenized_train_set = train_set.map(
    lambda it: tokenizer(it['text'], truncation=True),
    batched=True, batch_size=MINIBATCH_SIZE
)

tokenized_validation_set = validation_set.map(
    lambda it: tokenizer(it['text'], truncation=True),
    batched=True, batch_size=MINIBATCH_SIZE
)

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

In [135]:
print(tokenized_train_set)

Dataset({
    features: ['index_id', 'category', 'text', 'input_ids', 'attention_mask'],
    num_rows: 701
})


In [136]:
cls_metric = evaluate.load('f1')

In [137]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return cls_metric.compute(predictions=predictions, references=labels, average='macro')

In [138]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [139]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))

indices_of_categories = list(range(len(list_of_categories)))
n_categories = len(list_of_categories)
print(f'Categories for classification are: {list_of_categories}')
id2label = dict(zip(indices_of_categories, list_of_categories))
label2id = dict(zip(list_of_categories, indices_of_categories))

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [140]:
labeled_train_set = tokenized_train_set.add_column(
    'label',
    [label2id[val] for val in tokenized_train_set['category']]
)

labeled_validation_set = tokenized_validation_set.add_column(
    'label',
    [label2id[val] for val in tokenized_validation_set['category']]
)

In [141]:
print(labeled_train_set)

Dataset({
    features: ['index_id', 'category', 'text', 'input_ids', 'attention_mask', 'label'],
    num_rows: 701
})


In [142]:
classifier = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=n_categories, id2label=id2label, label2id=label2id
).cuda()

for param in classifier.parameters(): param.data = param.data.contiguous()

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at deepvk/USER-bge-m3 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [143]:
training_args = TrainingArguments(
    output_dir='rubert_sib200',
    learning_rate=5e-5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=10,
    weight_decay=1e-3,
    eval_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2,
    load_best_model_at_end=True
)

In [144]:
trainer = Trainer(
    model=classifier,
    args=training_args,
    train_dataset=labeled_train_set,
    eval_dataset=labeled_validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [146]:
for param in classifier.base_model.parameters():
    param.requires_grad = False

In [147]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,1.373227,0.433938
2,No log,1.057799,0.675560
3,No log,0.872295,0.778872
4,No log,0.765750,0.803501
5,No log,0.692464,0.813720
6,No log,0.649992,0.835983
7,No log,0.623123,0.835983
8,No log,0.602988,0.828074
9,No log,0.591999,0.828074
10,No log,0.588920,0.828074


TrainOutput(global_step=440, training_loss=0.7218847101384943, metrics={'train_runtime': 91.4571, 'train_samples_per_second': 76.648, 'train_steps_per_second': 4.811, 'total_flos': 825632328724848.0, 'train_loss': 0.7218847101384943, 'epoch': 10.0})

In [148]:
training_args = TrainingArguments(
    output_dir='rubert_sib200',
    learning_rate=2e-6,
    warmup_ratio=0.5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=10,
    weight_decay=1e-3,
    eval_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2,
    load_best_model_at_end=True
)

In [149]:
trainer = Trainer(
    model=classifier,
    args=training_args,
    train_dataset=labeled_train_set,
    eval_dataset=labeled_validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [150]:
for param in classifier.base_model.parameters():
    param.requires_grad = True

In [151]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.574299,0.839917
2,No log,0.537906,0.839917
3,No log,0.493532,0.848604
4,No log,0.459381,0.850833
5,No log,0.417928,0.827751
6,No log,0.407921,0.833142
7,No log,0.387299,0.827751
8,No log,0.380902,0.838607
9,No log,0.375520,0.857822
10,No log,0.376699,0.857822


TrainOutput(global_step=440, training_loss=0.19948061162775213, metrics={'train_runtime': 260.5267, 'train_samples_per_second': 26.907, 'train_steps_per_second': 1.689, 'total_flos': 825632328724848.0, 'train_loss': 0.19948061162775213, 'epoch': 10.0})

In [152]:
classifiсation_pipeline = pipeline('text-classification', model=classifier, tokenizer=tokenizer, device=0)

Без дополнительных данных

In [153]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(validation_set['text'])))
y_true = validation_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       0.67      0.67      0.67         9
         geography       0.80      1.00      0.89         8
            health       1.00      0.82      0.90        11
          politics       0.93      0.93      0.93        14
science/technology       0.93      1.00      0.96        25
            sports       1.00      0.83      0.91        12
            travel       0.75      0.75      0.75        20

          accuracy                           0.87        99
         macro avg       0.87      0.86      0.86        99
      weighted avg       0.87      0.87      0.87        99



In [154]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(test_set['text'])))
y_true = test_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       0.87      0.68      0.76        19
         geography       0.87      0.76      0.81        17
            health       1.00      0.95      0.98        22
          politics       0.94      0.97      0.95        30
science/technology       0.91      0.98      0.94        51
            sports       0.92      0.92      0.92        25
            travel       0.90      0.95      0.93        40

          accuracy                           0.92       204
         macro avg       0.91      0.89      0.90       204
      weighted avg       0.92      0.92      0.91       204



С данными 

In [130]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(validation_set['text'])))
y_true = validation_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       0.60      0.67      0.63         9
         geography       0.88      0.88      0.88         8
            health       0.90      0.82      0.86        11
          politics       0.82      1.00      0.90        14
science/technology       0.88      0.88      0.88        25
            sports       1.00      0.92      0.96        12
            travel       0.72      0.65      0.68        20

          accuracy                           0.83        99
         macro avg       0.83      0.83      0.83        99
      weighted avg       0.83      0.83      0.83        99



In [131]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(test_set['text'])))
y_true = test_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       0.94      0.79      0.86        19
         geography       0.88      0.82      0.85        17
            health       1.00      0.77      0.87        22
          politics       0.93      0.87      0.90        30
science/technology       0.86      0.96      0.91        51
            sports       0.83      0.96      0.89        25
            travel       0.93      0.95      0.94        40

          accuracy                           0.90       204
         macro avg       0.91      0.87      0.89       204
      weighted avg       0.90      0.90      0.90       204

